In [0]:
import scandir
import os
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import scipy as sp
import dill
import random
import cyvcf
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
r = ro.r

In [0]:
analysis_dir = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/notimputed/"
snp_file_gz = "isect.vcf.gz.sorted.gz"

## write piMASS files

In [0]:
hdf = HDFStoreHelper(os.path.join(analysis_dir, "isect.hd5"))
hdf_all = HDFStoreHelper(os.path.join(analysis_dir, "gypsy_samtools12_snps.vcf.gz.hd5"))

In [0]:
hdf_all.get_group_names()

In [0]:
hdf.get_group_names()

In [0]:
pimass_pheno = hdf_all.get("pca_std_pheno")[["Population",
                                             "Number",
                                             "Mass","Pupual Duration", "Total Dev Time"]]
pimass_pheno.head()

In [0]:
pca_x = hdf.get('pca_x')
pca_x.head()

In [0]:
pca_std_pheno = pimass_pheno.join(pca_x, how="inner")

In [0]:
pca_std_pheno.head()

In [0]:
pimass_pheno_pca = pca_std_pheno[[x for x in pca_std_pheno if "PC" in x or 'Mass' in x or 'Pupual' in x or 'Total Dev' in x]]
pimass_pheno_pca.columns = [x.replace(" ", "_") for x in pimass_pheno_pca.columns]
pimass_pheno_pca.index = [x for x in pimass_pheno_pca.index]
phenos = ["Mass", "Pupual_Duration", "Total_Dev_Time"]
for p in phenos:
    mod = smf.ols(formula="%s~PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8" % p, data=pimass_pheno_pca)
    res = mod.fit()
    col = "%s_resid" % p
    col = col.lower()
    pimass_pheno[col] = res.resid

In [0]:
z12_swapped = hdf.get("z12_swapped")

In [0]:
z12_swapped.head()

In [0]:
translation_df = pd.read_csv("translation_table.csv", sep="\t", index_col=0)

def get_correct_name(row, trans):
    trans[row.name] = "%s_%d_%d" % (row['pop'], row.indiv, row.dup)

name_translation = {}
translation_df.apply(get_correct_name, args=(name_translation,), axis=1);

In [0]:
readvcf = open(os.path.join(analysis_dir, snp_file_gz))
reader = cyvcf.VCFReader(readvcf)
gt_base_data = {}
gt_ref_alt = {}
at = 0
for snp in reader:
    snp_id = "%s_%d" % (snp.CHROM, snp.POS)
    gt_ref_alt[snp_id] = {'ref': snp.REF, 'alt': snp.ALT[0]}
    for sample in snp.samples:
        if not snp_id in gt_base_data:
            gt_base_data[snp_id] = {}
        sample_name = name_translation[sample.sample]
        bases = sample.gt_bases
        gt_base_data[snp_id][sample_name] = bases
    at += 1
    if at % 1000 == 0:
        print at
gt_base_df = pd.DataFrame(gt_base_data)
readvcf.close()

In [0]:
gt_base_df.head()

In [0]:
def swap_gt_alleles(gt, het):   
    if isinstance(gt, float): #NaN
        return np.NaN
    if gt is None:
        return np.NaN
    if gt[0] == gt[-1]:
        return gt
    else:
        return het # already in minor/major
    
def swap_gt(snp):
    vc = snp.value_counts()
    counts = {}
    for v in vc.index:
        if not v[0] in counts:
            counts[v[0]] = 0.0
        if not v[-1] in counts:
            counts[v[-1]] = 0.0
        counts[v[0]] += vc[v]
        counts[v[-1]] += vc[v]
    counts2 = sorted(counts.items(), key=operator.itemgetter(1)) #e.g., [('A', 110.0), ('G', 236.0)]
    minor = counts2[0][0]
    major = counts2[1][0]
    het = "%s/%s" % (minor, major)
    gt_ref_alt[snp.name]['minor'] = minor
    gt_ref_alt[snp.name]['major'] = major
    return snp.apply(swap_gt_alleles, args=(het,))
gt_base_df_swapped = gt_base_df.apply(swap_gt)
gt_base_df_swapped.head()

In [0]:
def convert_GQ_to_p(q):
    return pow(10,(q/-10.0))

def get_dosage(GP):
    total = 0
    if GP is None:
        return "NA"
    if sum(GP) == 0:
        return "NA"
    else:
        pvals = [convert_GQ_to_p(x) for x in GP]
        pval_sum = np.sum(pvals)
        pvals = [x/pval_sum for x in pvals]
        for i, val in enumerate(pvals):
            total += val*i
            
    return np.round(total, 3)

def get_GP(sample, flip):
    if sample['GT'] is None:
        return None, [0,0,0], ""
    if flip:
        return sample['GT'], sample['GP'][::-1], "flipped"
    else:
        return sample['GT'], sample['GP'], ""
        

def get_major_minor(snp, reader):
    d = snp.name.split("_")
    loc = int(d[-1])
    contig = "_".join(d[0:-1])
    minor_major = "%s%s" % (gt_ref_alt[snp.name]['minor'], gt_ref_alt[snp.name]['major'])
    ref_alt = "%s%s" % (gt_ref_alt[snp.name]['ref'], gt_ref_alt[snp.name]['alt'])
    flip = False
    if minor_major != ref_alt:
        flip = True
    dosages = []
    samples = []
    thesnp = list(reader.fetch(contig, loc, loc))[0]
    for sample in thesnp.samples:
        gt, gp, flipped = get_GP(sample, flip)
        dosages.append(get_dosage(gp))
        samples.append(sample.sample)
    data = [minor_major[0], minor_major[1]]
    index = ["minor", "major"]
    index.extend(samples)
    data.extend(dosages)
    ret = pd.Series(data, index=index)
    return ret
h = open(os.path.join(analysis_dir, snp_file_gz))
reader = cyvcf.VCFReader(h)
pimass_gt = gt_base_df_swapped.apply(get_major_minor, args=(reader,)).T

In [0]:
h.close()

In [0]:
pimass_gt.head()

In [0]:
pimass_pheno = pimass_pheno.reindex(index=gt_base_df.index)

In [0]:
%R -i pimass_pheno

In [0]:
%%R
massx = qqnorm(pimass_pheno$mass_resid, plot.it=F)$x
tdtx = qqnorm(pimass_pheno$total_dev_time_resid, plot.it=F)$x
pdx = qqnorm(pimass_pheno$pupual_duration_resid, plot.it=F)$x

In [0]:
pimass_pheno['massx'] = r('massx')
pimass_pheno['tdtx'] = r('tdtx')
pimass_pheno['pdx'] = r('pdx')

In [0]:
pimass_pheno.head()

In [0]:
pimass_pheno.massx.to_csv(os.path.join(analysis_dir, "pimass_mass.txt"),
                                     index=False,
                                     header=False)
pimass_pheno.tdtx.to_csv(os.path.join(analysis_dir, "pimass_tdt.txt"),
                                     index=False,
                                     header=False)
pimass_pheno.pdx.to_csv(os.path.join(analysis_dir, "pimass_pd.txt"),
                                     index=False,
                                     header=False)
pimass_pheno.to_csv(os.path.join(analysis_dir, "pimass_pheno.txt"),
                                     index=True,
                                     header=True)

In [0]:
pimass_contigs = {}
with open(os.path.join(analysis_dir, "pimass_loc.txt"), "w") as o:    
    for x in pimass_gt.index:
        data = x.split("_")
        contig = "_".join(data[0:-1])
        pos = data[-1]
        if not contig in pimass_contigs:
            pimass_contigs[contig] = []
        pimass_contigs[contig].append(pos)
    
    chrom_id = 1
    for contig, positions in pimass_contigs.items():
        for p in positions:
            o.write("%s_%s\t%s\t%d\n" % (contig, p, p, chrom_id))
        chrom_id += 1

In [0]:
import random
def create_pimass_run_files(num_runs):
    phenos = ["mass", 'tdt', 'pd']
    for p in phenos:
        with open(os.path.join(analysis_dir, "pimass_%s_run.txt" % p), "w") as o:
            for i in xrange(num_runs):
                cmd = "~/g/src/pimass/pimass-lin \
-g pimass_gt.txt \
-p pimass_%s.txt -pos pimass_loc.txt \
-o pimass_%s_out_%d \
-w 1000000 \
-s 10000000 \
-num 500 \
-smin 1 \
-smax 100 \
-hmin 0.01 \
-hmax 0.9 \
-pmin 1 \
-pmax 1000 \
-r %.0f" % (p, p, i, int(random.getrandbits(32)))
                o.write("%s\n" % cmd) 
                


def create_qsub_files():
    files = !ls {analysis_dir}*run.txt
    for f in files:
        with open("%s_qsub.sh" % f, "w") as o:
            o.write("""#!/bin/bash
#$ -j y
#$ -V
#$ -N pimass_%s
#$ -cwd
parallel -a %s
""" % (os.path.basename(f).split("_")[1], f))
            
create_pimass_run_files(10)
create_qsub_files()

## run piMASS

```bash
./run_pimass.sh
mv output output_comeault_isect
```



In [0]:
assembly = "/home/cfriedline/gpfs/assemblies/gypsy/masurca_new/CA/10-gapclose/genome.ctg.fasta"

In [0]:
filedir = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/notimputed/output_comeault_isect/"

In [0]:
def dump_session():
    dill.settings['recurse'] = True
    dill.settings['fmode'] = dill.HANDLE_FMODE
    dill.dump_session(filename=os.path.join(filedir, "pimass.dill"))

In [0]:
path_files = {}
mcmc_files = {}
gamma_files = {}
snp_files = {}
for root, dirs, files in scandir.walk(filedir):
    for f in files:
        d = f.split("_")
        pheno = d[1]
        if not pheno in path_files:
            path_files[pheno] = []
            mcmc_files[pheno]= []
            gamma_files[pheno] = []
            snp_files[pheno] = []
        if 'path' in f:
            path_files[pheno].append(os.path.join(root, f))
        elif 'mcmc' in f:
            mcmc_files[pheno].append(os.path.join(root, f))
        elif 'gamma' in f:
            gamma_files[pheno].append(os.path.join(root, f))
        elif 'snp' in f:
            snp_files[pheno].append(os.path.join(root, f))

In [0]:
%%R
library(coda)

In [0]:
mcmc = r('mcmc')
mcmc_list = r('mcmc.list')

In [0]:
dfs = {}
phenos = ["mass", "pd", "tdt"]
for pheno in phenos:
    frames = [pd.read_csv(x,sep="\t") for x in path_files[pheno]]
    frames = [x.ix[:,:-1] for x in frames]
    for df in frames:
        df.columns = [x.strip() for x in df.columns]
    dfs[pheno] = frames

In [0]:
dfs['mass'][0].head()

In [0]:
path_mcmc_r = {}
path_mcmc = {}
thin = 1
for key, dflist in dfs.items():
    path_mcmc_r[key] = [mcmc(pandas2ri.DataFrame(x.sample(frac=thin).sort_index())) for x in dflist]
    path_mcmc[key] = [x.sample(frac=thin).sort_index() for x in dflist]

In [0]:
path_mcmc_list_mass = mcmc_list(path_mcmc_r['mass'])
path_mcmc_list_pd = mcmc_list(path_mcmc_r['pd'])
path_mcmc_list_tdt = mcmc_list(path_mcmc_r['tdt'])

In [0]:
%R -i path_mcmc_list_mass -i path_mcmc_list_pd -i path_mcmc_list_tdt

In [0]:
%%R
effective_sizes_mass = lapply(path_mcmc_list_mass,effectiveSize)
effective_sizes_pd = lapply(path_mcmc_list_pd,effectiveSize)
effective_sizes_tdt = lapply(path_mcmc_list_tdt,effectiveSize)

In [0]:
def get_effective_sizes(r_name):
    df = pd.DataFrame([pandas2ri.ri2py(x) for x in r[r_name]])
    test = r[r_name].rx2(1)
    df.columns = r('names')(test)
    return df
ne_tdt = get_effective_sizes('effective_sizes_tdt')
ne_pd= get_effective_sizes('effective_sizes_pd')
ne_mass= get_effective_sizes('effective_sizes_mass')


In [0]:
print ne_tdt.mean()
print ne_tdt.std()


In [0]:
ne_pd.mean()

In [0]:
ne_mass.mean()

In [0]:
print "MASS", r("summary")(path_mcmc_list_mass)
print "PD", r("summary")(path_mcmc_list_pd)
print "TDT", r("summary")(path_mcmc_list_tdt)

In [0]:
%%R
plot(path_mcmc_list_mass)
plot(path_mcmc_list_pd)
plot(path_mcmc_list_tdt)

In [0]:
mcmc = {}
for pheno, files in mcmc_files.items():
    if not pheno in mcmc:
        mcmc[pheno] = pd.DataFrame()
    for f in files:
        index = os.path.basename(f).split("_")[-1].split(".")[0]
        testdf = pd.read_csv(f, sep="\t")
        testdf.columns = ["%s_%s" % (x.strip(), index) for x in testdf.columns]
        mcmc[pheno] = pd.concat([mcmc[pheno], testdf], axis=1)

In [0]:
mcmc_mass = mcmc['mass']
mcmc_pd = mcmc['pd']
mcmc_tdt = mcmc['tdt']

In [0]:
def percent_difference(x, y):
    x = float(x)
    y = float(y)
    return (np.abs(x-y)/np.mean([x, y]))*100

def get_quantile_max(name, data, q):
    d = data.quantile(q)
    d.index = [str(x) for x in d.index]
    d['median_val'] = data.median()
    d['mean_val'] = data.mean()
    d['cutoff'] = 0.01
    d["x99_cutoff"] = percent_difference(d['0.99'], d['cutoff'])
    d["x99_median"] =  percent_difference(d['0.99'], d['median_val'])
    d["x95_cutoff"] = percent_difference(d['0.95'], d['cutoff'])
    d["x95_median"] =  percent_difference(d['0.95'], d['median_val'])
    d['relaxed_cutoff'] = d['0.99']
    d['min'] = data.min()
    d['max'] = data.max()
    d.name = name
    return d

mass_quant = get_quantile_max("mass", mcmc_mass.postrb_0, [0.95,0.99])
pd_quant = get_quantile_max("pd", mcmc_pd.postrb_0, [0.95,0.99])
tdt_quant =get_quantile_max("tdt", mcmc_tdt.postrb_0, [0.95,0.99]) 

In [0]:
print "%s\n\n%s\n\n%s\n" % (mass_quant, pd_quant, tdt_quant)

In [0]:
hdf.put('mass_quant', mass_quant)
hdf.put('pd_quant', pd_quant)
hdf.put('tdt_quant', tdt_quant)

In [0]:
sig_snps_mass = mcmc_mass[mcmc_mass.postrb_0 > mass_quant.cutoff]
sig_snps_tdt = mcmc_tdt[mcmc_tdt.postrb_0 > tdt_quant.cutoff]
sig_snps_pd = mcmc_pd[mcmc_pd.postrb_0 > pd_quant.cutoff]

relaxed_sig_snps_mass = mcmc_mass[mcmc_mass.postrb_0 > mass_quant.relaxed_cutoff]
relaxed_sig_snps_tdt = mcmc_tdt[mcmc_tdt.postrb_0 > tdt_quant.relaxed_cutoff]
relaxed_sig_snps_pd = mcmc_pd[mcmc_pd.postrb_0 > pd_quant.relaxed_cutoff]

In [0]:
contig_pips = {}
def get_contig_pip(row, pheno):
    if not pheno in contig_pips:
        contig_pips[pheno] = {}
        
    d = row.rs_1.split("_")
    contig = "_".join(d[:-1])
    if not contig in contig_pips[pheno]:
        contig_pips[pheno][contig] = {'postc':0,
                              'beta':0,
                              'betarb':0,
                              'postrb':0}
    contig_pips[pheno][contig]['postc'] += row.postc_1
    contig_pips[pheno][contig]['postrb'] += row.postrb_1
    contig_pips[pheno][contig]['beta'] += row.beta_1
    contig_pips[pheno][contig]['betarb'] += row.betarb_1

for pheno, df in mcmc.items():
    print pheno
    df.apply(get_contig_pip, args=(pheno,), axis=1)


In [0]:
contig_pip_dfs = {}
for pheno, data in contig_pips.items():
    contig_pip_dfs[pheno] = pd.DataFrame(data).T

In [0]:
contig_pip_dfs

In [0]:
from Bio import SeqIO
contig_lengths = {}
for rec in SeqIO.parse(assembly,"fasta"):
    contig_lengths[rec.name] = {"length":len(rec)}

In [0]:
contig_length_df = pd.DataFrame(contig_lengths).T

In [0]:
contig_length_df.head()

In [0]:
contig_pip_mass = contig_pip_dfs['mass'].join(contig_length_df)

In [0]:
plt.plot(contig_pip_dfs['tdt'].postrb, label="PIP")
plt.legend()
plt.show()

In [0]:
plt.xlim(0, len(mcmc_mass))
#plt.plot(avg_pip_mass, label="PIP")
#plt.plot(avg_effect_mass, alpha=0.5, label="Beta")
plt.plot(avg_pip_mass_rb, alpha=0.5, label="PIP (RB)")
plt.plot(avg_effect_mass_rb, alpha=0.5, label="Beta (RB)")
plt.title("Mass")
plt.xlabel("SNP")
plt.legend()
plt.show()

In [0]:
plt.xlim(0, len(mcmc_pd))
#plt.plot(avg_pip_pd, label="PIP")
#plt.plot(avg_effect_pd, alpha=0.5, label="Beta")
plt.plot(avg_pip_pd_rb, alpha=0.5, label="PIP (RB)")
plt.plot(avg_effect_pd_rb, alpha=0.5, label="Beta (RB)")
plt.title("PD")
plt.xlabel("SNP")
plt.legend()
plt.show()

In [0]:
plt.xlim(0, len(mcmc_tdt))
#plt.plot(avg_pip_tdt, label="PIP")
#plt.plot(avg_effect_tdt, alpha=0.5, label="Beta")
plt.plot(avg_pip_tdt_rb, alpha=0.5, label="PIP (RB)")
plt.plot(avg_effect_tdt_rb, alpha=0.5, label="Beta (RB)")
plt.title("TDT")
plt.xlabel("SNP")
plt.legend()
plt.show()

In [0]:
mcmc_pd.head()

In [0]:
snps = {}
for pheno, files in snp_files.items():
    if not pheno in snps:
        snps[pheno] = pd.DataFrame()
    for f in files:
        index = os.path.basename(f).split("_")[-1].split(".")[0]
        h = open(f)
        h.readline() ##skip header
        header = h.readline().strip().split()
        data = []
        for line in h:
            line = line.strip().split()
            data.append(line)
            
        testdf = pd.DataFrame(data, columns=header)
        testdf.columns = ["%s_%s" % (x.strip(), index) for x in testdf.columns]
        snps[pheno] = pd.concat([snps[pheno], testdf], axis=1)

In [0]:
snps_mass = snps['mass'][[x for x in snps['mass'] if '_1' in x]]

In [0]:
snps_mass.head()

In [0]:
def read_gamma(f):
    d = []
    h = open(f)
    header = h.readline().strip().split()
    for line in h:
        line = line.strip().split()
        d.append(line)
    df = pd.DataFrame(d, columns=header)
    return df.replace('NA', np.nan).astype(float)
gamma_mass = read_gamma(gamma_files['mass'][0])
gamma_pd = read_gamma(gamma_files['pd'][0])
gamma_tdt = read_gamma(gamma_files['tdt'][0])

In [0]:
snp_density = {}
def get_snp_density(row):
    included = row[1:].dropna()
    for snp_id in included:
        if not snp_id in snp_density:
            snp_density[snp_id] = 0
        snp_density[snp_id] += 1
    
gamma_mass.apply(get_snp_density, axis=1);

In [0]:
sig_snps_mass.head()

In [0]:
def get_hmean(df, col_pattern):
    cols = ['rs','chr']
    cols.extend(["%s_hmean" % x for x in col_pattern])
    d = pd.DataFrame(columns=cols, index=df.index)
    d['rs'] = df.rs_1.values
    d["chr"] = df.chr_1.values
    for cp in col_pattern:
        #np. abs here to account for negative betas
        d["%s_hmean" % cp] = df[[x for x in df if cp in x]].apply(np.abs, axis=1).apply(sp.stats.hmean, axis=1).values
    return d
sig_snps_mass_hmean = get_hmean(sig_snps_mass, ["postrb", "betarb"])
sig_snps_tdt_hmean = get_hmean(sig_snps_tdt, ["postrb", "betarb"])
sig_snps_pd_hmean = get_hmean(sig_snps_pd, ["postrb", "betarb"])
relaxed_sig_snps_mass_hmean = get_hmean(relaxed_sig_snps_mass, ["postrb", "betarb"])
relaxed_sig_snps_tdt_hmean = get_hmean(relaxed_sig_snps_tdt, ["postrb", "betarb"])
relaxed_sig_snps_pd_hmean = get_hmean(relaxed_sig_snps_pd, ["postrb", "betarb"])

In [0]:
hdf.put("sig_snps_mass", sig_snps_mass)
hdf.put("sig_snps_tdt", sig_snps_tdt)
hdf.put("sig_snps_pd", sig_snps_pd)
hdf.put("relaxed_sig_snps_mass", relaxed_sig_snps_mass)
hdf.put("relaxed_sig_snps_tdt", relaxed_sig_snps_tdt)
hdf.put("relaxed_sig_snps_pd", relaxed_sig_snps_pd)

hdf.put("sig_snps_mass_hmean", sig_snps_mass_hmean)
hdf.put("sig_snps_tdt_hmean", sig_snps_tdt_hmean)
hdf.put("sig_snps_pd_hmean", sig_snps_pd_hmean)
hdf.put("relaxed_sig_snps_mass_hmean", relaxed_sig_snps_mass_hmean)
hdf.put("relaxed_sig_snps_tdt_hmean", relaxed_sig_snps_tdt_hmean)
hdf.put("relaxed_sig_snps_pd_hmean", relaxed_sig_snps_pd_hmean)

In [0]:
plt.hist(np.abs(sig_snps_mass.betarb_0.values))
plt.text(0.010, 50, r"$n = %d$" % len(sig_snps_mass))
plt.title(r"Mass ($> %.2f$)" % mass_quant.cutoff)
plt.xlabel(r"$\beta$")
plt.show()
plt.hist(np.abs(relaxed_sig_snps_mass.betarb_0.values))
plt.text(0.01, 70, r"$n = %d$" % len(relaxed_sig_snps_mass))
plt.title(r"Mass 99th($> %.5f$)" % mass_quant.relaxed_cutoff)
plt.xlabel(r"$\beta$")
plt.show()

In [0]:
plt.hist(np.abs(sig_snps_tdt.betarb_0.values))
plt.text(0.015, 12, r"$n = %d$" % len(sig_snps_tdt))
plt.title(r"TDT ($> %.2f$)" % tdt_quant.cutoff)
plt.xlabel(r"$\beta$")
plt.show()
plt.hist(np.abs(relaxed_sig_snps_tdt.betarb_0.values))
plt.text(0.015, 100, r"$n = %d$" % len(relaxed_sig_snps_tdt))
plt.title(r"TDT 99th ($> %.5f$)" % tdt_quant.relaxed_cutoff)
plt.xlabel(r"$\beta$")
plt.show()

In [0]:
plt.hist(np.abs(sig_snps_pd.betarb_0.values))
plt.text(0.0045, 3, r"$n = %d$" % len(sig_snps_pd))
plt.title(r"PD ($> %.2f$)" % pd_quant.cutoff)
plt.xlabel(r"$\beta$")
plt.show()
plt.hist(np.abs(relaxed_sig_snps_pd.betarb_0.values))
plt.text(0.0045, 70, r"$n = %d$" % len(relaxed_sig_snps_pd))
plt.title(r"PD 99th ($> %.5f$)" % pd_quant.relaxed_cutoff)
plt.xlabel(r"$\beta$")
plt.show()